In [2]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import cv2,math,gc

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.nn import Parameter
import torch.optim as optim

!pip install "../input/efficient-net/dist/efficientnet_pytorch-0.7.0.tar"
from efficientnet_pytorch import EfficientNet

!pip install "../input/faissgpuwheel/faiss_gpu-1.7.0-cp37-cp37m-manylinux2014_x86_64.whl"
import faiss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

import warnings
warnings.simplefilter('ignore')
    
torch.backends.cudnn.benchmark = True
from transformers import (BertTokenizer, BertModel,
                          DistilBertTokenizer, DistilBertModel)
from sklearn.preprocessing import LabelEncoder
from tqdm.autonotebook import tqdm

Processing /kaggle/input/efficient-net/dist/efficientnet_pytorch-0.7.0.tar
DEPRECATION: Source distribution is being reinstalled despite an installed package having the same name and version as the installed package. pip 21.1 will remove support for this functionality. A possible replacement is use --force-reinstall. You can find discussion regarding this at https://github.com/pypa/pip/issues/8711.
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16033 sha256=e4f5c2d876a6fc64c04676d5c4091ab9c8902456b948a59001ae4565aa794eb6
  Stored in directory: /root/.cache/pip/wheels/af/8c/80/1bf8cc2fa471c320978f34c5290675daaa96446e1b9ba45555
Successfully built efficientnet-pytorch
  Attempting uninstall: efficientnet-pytorch
    Found existing installation: efficientnet-pytorch 0.7.0
    Uninstalling efficientnet-pytorch-0.7.0:
      Successfully uninstalled efficientnet-pytorch-0.7.0
Processing /kaggle/input/faissgpuwheel/faiss_gpu-1.7.0-cp37-cp37m

In [4]:
TRAIN = False

In [5]:
class cfg:
    img_size = (380,380)
    feavec_num1 = 512
    feavec_num2 = 768
    fea_norm = 64
    margin = 0.35
    batch = 16
    mname = 'efficientnet-b3'
    clsize = 8812
    lr = 0.001
    momentum = 0.9
    weight_decay = 0.0005
    log_interval = 1000
    epochs = 10
    wpath = '../input/my-weight/efficientnet-b3_eff_bert_arcface_epoch_10.pt'

In [6]:
class CFG:
    DistilBERT = True # if set to False, BERT model will be used
    bert_hidden_size = 768
    
    batch_size = 64
    epochs = 100
    num_workers = 4
    learning_rate = 1e-5 
    scheduler = "ReduceLROnPlateau"
    step = 'epoch'
    patience = 2
    factor = 0.8
    dropout = 0.5
    model_path = "/kaggle/working"
    max_length = 30
    model_save_name = "bert_model.pt"
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [7]:
df = pd.read_csv("../input/shopee-product-matching/test.csv")
    
df_cu = cudf.DataFrame(df)

if len(df)==3:
    cfg.batch = 3
print('df shape is', df.shape )
df.head()

df shape is (3, 4)


,posting_id,image,image_phash,title
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng


In [8]:
if CFG.DistilBERT:
    model_name='cahya/distilbert-base-indonesian'
    tokenizer = DistilBertTokenizer.from_pretrained("../input/distilbertbaseindonesianfte1")
    bert_model = DistilBertModel.from_pretrained("../input/distilbertbaseindonesianfte1")
else:
    model_name='cahya/bert-base-indonesian-522M'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    bert_model = BertModel.from_pretrained(model_name)

In [9]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.30, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output  #need softmax then


class Model(nn.Module):
    def __init__(self,name,clustersize,feavec, bert_model, last_hidden_size=CFG.bert_hidden_size):
        super(Model, self).__init__()
        self.eff = EfficientNet.from_name(name)
        self.bert_model = bert_model
        self.out = nn.Linear(1000+last_hidden_size,feavec)
        self.margin = ArcMarginProduct(in_features=feavec, 
                                       out_features = clustersize, 
                                       s=cfg.fea_norm, 
                                       m=cfg.margin)      
    
    def get_bert_features(self, batch):
        output = self.bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        last_hidden_state = output.last_hidden_state # shape: (batch_size, seq_length, bert_hidden_dim)
        CLS_token_state = last_hidden_state[:, 0, :] # obtaining CLS token state which is the first token.
        return CLS_token_state

    def forward(self, batch, labels=None):
        x1 = self.eff(batch['image'])
        x2 = self.get_bert_features(batch)
        
        x = torch.hstack((x1, x2))
        x = self.out(x)
        if labels is not None:
            return self.margin(x,labels)
        return F.normalize(x,dim=1)

In [10]:
model = Model(name=cfg.mname,clustersize=cfg.clsize, feavec=512, bert_model=bert_model).to(device)
model.load_state_dict(torch.load(cfg.wpath, map_location=device))

<All keys matched successfully>

In [11]:
def load_image(file_name):
    if TRAIN:
        file_path = f'/kaggle/input/shopee-product-matching/train_images/{file_name}'
    else:
        file_path = f'/kaggle/input/shopee-product-matching/test_images/{file_name}'
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, cfg.img_size)
    tensor_img = torch.tensor(img)
    tensor_img = tensor_img.permute(( 2, 0, 1)).float()/255.0
    return tensor_img

In [12]:
class MyDataset(Dataset):
    def __init__(self, dataframe, tokenizer, mode="train", max_length=None):
        self.dataframe = dataframe
        if mode != "test":
            self.targets = dataframe['label'].values
        texts = list(dataframe['title'].apply(lambda o: str(o)).values)
        self.encodings = tokenizer(texts, 
                                   padding=True, 
                                   truncation=True, 
                                   max_length=max_length)
        self.mode = mode
        self.img = dataframe.image.values
        
    def __getitem__(self, idx):
        # putting each tensor in front of the corresponding key from the tokenizer
        # HuggingFace tokenizers give you whatever you need to feed to the corresponding model
        item = {key: torch.tensor(values[idx]) for key, values in self.encodings.items()}
        # when testing, there are no targets so we won't do the following
        if self.mode != "test":
            item['labels'] = torch.tensor(self.targets[idx]).long()
        img = self.img[idx]
        img = load_image(img)
        item['image'] = img
        return item
    
    def __len__(self):
        return len(self.dataframe)

In [13]:
def all_embeddings(df):
    dataset = MyDataset(df, tokenizer, mode='test', max_length=CFG.max_length)
    loader = DataLoader(dataset,
                        batch_size=cfg.batch,
                        shuffle=False,
                        num_workers=2,
                        pin_memory=True,
                        drop_last=False)
    
    model.eval()
    print('start collection')
    feavec = 512
    embedded1 = np.empty((0,feavec),dtype='float32')
    with torch.no_grad():
        for idx,batch in enumerate(loader):
            batch = {k: v.to(CFG.device) for k, v in batch.items()}
            outputs = model(batch)
            embedded1 = np.append(embedded1, outputs.cpu().detach().numpy(),axis=0)

            if idx%100==0:
                print(idx,len(loader))
                print(embedded1.shape)
    return embedded1

In [14]:
def predict_img(df,embeddings,topk=50,threshold=0.63):
    N,D = embeddings.shape
    cpu_index = faiss.IndexFlatL2(D)
    gpu_index = faiss.index_cpu_to_all_gpus(cpu_index)
    gpu_index.add(embeddings)
    cluster_distance,cluster_index = gpu_index.search(x=embeddings, k=topk)
    
    df['pred_images'] = ''
    pred = []
    for k in range(embeddings.shape[0]):
        idx = np.where(cluster_distance[k,] < threshold)[0]
        ids = cluster_index[k,idx]
        #posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        posting_ids = df['posting_id'].iloc[ids].values
        pred.append(posting_ids)
    df['pred_images'] = pred
    df['pred_images'] = df.pred_images.apply(lambda x: ' '.join(x))
    return df

In [15]:
all_embeddings1= all_embeddings(df)

start collection
0 1
(3, 512)


In [1]:
df = predict_img(df,all_embeddings1,topk=50,threshold=0.88)
df

NameError: name 'predict_img' is not defined

In [17]:
with open('submission.csv', 'w') as outf:
    print('posting_id,matches', file=outf)
    for i,(idnum,match) in enumerate(zip(df['posting_id'],df['pred_images'])):
        print(f'{idnum},{match}', file=outf)

In [18]:
df_t = pd.read_csv("submission.csv")
print(df_t)

        posting_id          matches
0  test_2255846744  test_2255846744
1  test_3588702337  test_3588702337
2  test_4015706929  test_4015706929
